# SolubilityCCS Example - Acid Formation Analysis

This notebook demonstrates how to use the SolubilityCCS package to analyze acid formation behavior in Carbon Capture and Storage (CCS) systems.

**Requirements:**
- Python 3.9+ 
- SolubilityCCS package with all dependencies

**What this example covers:**
- Setting up a fluid system with CO2, acid (H2SO4), and water
- Calculating phase behavior and acid formation risk
- Extracting CO2 thermodynamic properties

In [1]:
from solubilityccs import Fluid
from solubilityccs.neqsim_functions import get_co2_parameters

In [2]:
acid = "H2SO4"  # HNO3 or H2SO4
acid_in_co2 = 5.0  # ppm
water_in_co2 = 1  # ppm
temperature = 60  # C
pressure = 60  # bara
# Mt/year - flow rate for calculating liquid acid formation
flow_rate = 100

fluid = Fluid()
fluid.add_component("CO2", 1.0 - acid_in_co2 / 1e6 - water_in_co2 / 1e6)  # mole
fluid.add_component(acid, acid_in_co2 / 1e6)  # mole
fluid.add_component("H2O", water_in_co2 / 1e6)  # mole
fluid.set_temperature(temperature + 273.15)  # to Kelvin
fluid.set_pressure(pressure)  # bara
fluid.set_flow_rate(flow_rate * 1e6 * 1000 / (365 * 24), "kg/hr")
fluid.calc_vapour_pressure()
fluid.flash_activity()

print(
    "Mole fraction of gas phase to total phase", fluid.betta, "mol/mol"
)  # If this value is lower than 1 - danger of acid formation due to 2 phase
print(
    "water in CO2 ", 1e6 * fluid.phases[0].get_component_fraction("H2O"), " ppm mol"
)  # ppm
print(
    acid, "in CO2 ", 1e6 * fluid.phases[0].get_component_fraction(acid), " ppm mol"
)  # ppm

if fluid.betta < 1:
    print("Second phase is ", fluid.phases[1].name)
    print("Liquid phase formed", fluid.phases[1].get_acid_wt_prc(acid), " wt %")
    print(
        "Liquid phase formed",
        fluid.phases[1].get_flow_rate("kg/hr") * 24 * 365 / 1000,
        "t/y",
    )  # Flow rate needed for this calculation
    print(
        "Water in liquid phase ",
        fluid.phases[1].get_component_fraction("H2O"),
        "mol fraction",
    )  # ppm
    print(
        acid,
        " in liquid phase ",
        fluid.phases[1].get_component_fraction(acid),
        "mol fraction",
    )  # ppm

# Additional information for pure CO2 at this conditions

results = get_co2_parameters(pressure, temperature + 273.15)
print("Pure CO2 density", results["density"], "kg/m3")
print("Pure CO2 speed of sound:", results["speed_of_sound"], "m/s")
print("Pure CO2 enthalpy:", results["enthalpy"], "kJ/kg")
print("Pure CO2 entropy:", results["entropy"], "J/K")

Mole fraction of gas phase to total phase 0.999995957054125 mol/mol
water in CO2  0.6371814173480271  ppm mol
H2SO4 in CO2  1.3198818460084731  ppm mol
Second phase is  ACIDIC
Liquid phase formed 98.22138199748713  wt %
Liquid phase formed 834.3125631076986 t/y
Water in liquid phase  0.08974180708526193 mol fraction
H2SO4  in liquid phase  0.9102581929147381 mol fraction
Pure CO2 density 123.76922997021265 kg/m3
Pure CO2 speed of sound: 237.93838994219405 m/s
Pure CO2 enthalpy: 15.446075527066663 kJ/kg
Pure CO2 entropy: -29.420040334318706 J/K


In [3]:
# Using the ModelResults class for clean table output
from solubilityccs import ModelResults

# Create results object - note: 'results' here refers to the CO2 properties dict
co2_properties = get_co2_parameters(
    pressure, temperature + 273.15
)  # Save the CO2 properties dict (temperature in Kelvin)
results_obj = ModelResults(fluid, co2_properties=co2_properties)

# Display results as a formatted table
print(results_obj.generate_table())

═════════════════════════════════════════════════════════════════
                 SOLUBILITY CCS ANALYSIS RESULTS
═════════════════════════════════════════════════════════════════

📋 SYSTEM OVERVIEW
───────────────────────────────────
Acid Type:            H2SO4
Temperature:          60.0 °C
Pressure:             60.0 bara

⚗️  PHASE BEHAVIOR ASSESSMENT
───────────────────────────────────
🚨 STATUS: ACID FORMATION RISK DETECTED!
⚠️  RISK LEVEL: Two-phase system present

GAS PHASE COMPOSITION
───────────────────────────────────
Water in CO₂:         0.64 ppm (mol)
H2SO4 in CO₂:        1.32 ppm (mol)

LIQUID PHASE DETAILS
───────────────────────────────────
Phase Type:           ACIDIC
Acid Concentration:   98.221 wt%
Liquid Flow Rate:     834.312563 t/year
Water Mol Fraction: 0.089742
H2SO4 Mol Fraction: 0.910258

PURE CO₂ PROPERTIES
───────────────────────────────────
Density:              123.76922997021265 kg/m³
Speed of Sound:       237.93838994219405 m/s
Enthalpy:             15.44